In [1]:
%pip install -U langgraph langchain-tavily langgraph-checkpoint-sqlite python-dotenv langchain[openai]
%pip install -qU "langchain[openai]"

zsh:1: no matches found: langchain[openai]
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["LANGSMITH_TRACING"] = "true"


## Define a Search Tool

In [3]:
from langchain_tavily import TavilySearch

search = TavilySearch(max_results=2)
search_results = search.invoke("What is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

{'query': 'What is the weather in SF', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in San Francisco', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1757509126, 'localtime': '2025-09-10 05:58'}, 'current': {'last_updated_epoch': 1757508300, 'last_updated': '2025-09-10 05:45', 'temp_c': 17.8, 'temp_f': 64.0, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 4.0, 'wind_kph': 6.5, 'wind_degree': 243, 'wind_dir': 'WSW', 'pressure_mb': 1011.0, 'pressure_in': 29.86, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 87, 'cloud': 75, 'feelslike_c': 17.8, 'feelslike_f': 64.0, 'windchill_c': 15.9, 'windchill_f': 60.6, 'heatindex_c': 15.9, 'heatindex_f': 60.6, 'dewpoint_c': 15.3, 

## Using a LLM

In [4]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4.1", model_provider="openai")

### testing a query

In [5]:
query = "Hi!"
response = model.invoke([{"role": "user", "content": query}])
response.text()

'Hello! How can I help you today? 😊'

### adding tools

In [6]:
model_with_tools = model.bind_tools(tools)

In [7]:
query = "Hi!"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: Hello! How can I help you today?

Tool calls: []


In [8]:
query = "Search for the weather in SF"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: 

Tool calls: [{'name': 'tavily_search', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_sYg3o0h9sU8BF8YWZRDhCn2h', 'type': 'tool_call'}]


In [9]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

### Run the Agent

In [10]:
input_message = {"role": "user", "content": "Hi!"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi!
================================== Ai Message ==================================

Hello! How can I help you today?


In [11]:
input_message = {"role": "user", "content": "Search for the weather in SF"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Search for the weather in SF
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_oWySBUfcCuvyy6IEqlef4mto)
 Call ID: call_oWySBUfcCuvyy6IEqlef4mto
  Args:
    query: current weather in San Francisco
    search_depth: basic
================================= Tool Message =================================
Name: tavily_search

{"query": "current weather in San Francisco", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1757511417, 'localtime': '2025-09-10 06:36'}, 'current': {'last_updated_epoch': 1757511000, 'last_updated': '2025-09-10 06:30', 'temp_c': 1

In [12]:
for step in agent_executor.stream({"messages": [input_message]}, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Search for the weather in SF
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_pwxLhn4ahap6opZF9J5ch2gL)
 Call ID: call_pwxLhn4ahap6opZF9J5ch2gL
  Args:
    query: current weather in San Francisco
    search_depth: basic
================================= Tool Message =================================
Name: tavily_search

{"query": "current weather in San Francisco", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1757511417, 'localtime': '2025-09-10 06:36'}, 'current': {'last_updated_epoch': 1757511000, 'last_updated': '2025-09-10 06:30', 'temp_c': 1

In [13]:
for step, metadata in agent_executor.stream(
    {"messages": [input_message]}, stream_mode="messages"
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

Currently|,| in| San| Francisco|,| the| weather| is| partly| cloudy| with| a| temperature| of| about| |64|°F| (|18|°C|).| Hum|idity| is| high| at| |87|%,| and| there| is| a| light| southwest| wind| at| around| |3|.|6| mph|.| No| precipitation| is| reported|.

|For| updates| or| a| detailed| forecast|,| you| can| visit| a| weather| site| like| Weather|API|.|

### adding memory

In [14]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [15]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}


input_message = {"role": "user", "content": "Hi, I'm Bob!"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Bob!
================================== Ai Message ==================================

Hello, Bob! How can I assist you today?


In [16]:
input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

Your name is Bob! How can I help you today?


In [17]:
config = {"configurable": {"thread_id": "xyz123"}}

input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

You haven’t told me your name yet! If you’d like, please share your name and I’ll remember it for our conversation.
